In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
from tqdm import tqdm

# Load admissions data
admissions = pd.read_csv('/data/share/AKI/3.0/hosp/admissions.csv.gz', compression='gzip')

# Check for missing values and handle them
admissions.loc[admissions.admission_location.isnull(), 'admission_location'] = 'EMERGENCY ROOM'
admissions.loc[admissions.discharge_location.isnull(), 'discharge_location'] = 'HOME'
admissions.loc[admissions.insurance.isnull(), 'insurance'] = 'Medicare'
admissions.loc[admissions.marital_status.isnull(), 'marital_status'] = 'MARRIED'

# Add duration in hours
def convert_to_datetime(datestr: str):
    if ':' in datestr:
        date_time = datetime.datetime.strptime(datestr, '%Y-%m-%d %H:%M:%S')
    else:
        date_time = datetime.datetime.strptime(datestr, '%Y-%m-%d')
    return date_time

admissions['hrs'] = (
    admissions.dischtime.apply(lambda x: convert_to_datetime(x)) - 
    admissions.admittime.apply(lambda x: convert_to_datetime(x))
).apply(lambda x: round(x.seconds / 3600))

# Select columns for the new dataset
new_admissions = admissions.loc[:, [
    'subject_id', 'hadm_id', 'admission_type', 
    'admission_location', 'discharge_location', 
    'insurance', 'marital_status', 'race', 'hrs'
]]

# Save the cleaned admissions data
# os.mkdir('/data/share/AKI/3.0/hosp/new')
# new_admissions.to_csv('/data/share/AKI/3.0/hosp/new/admissions.csv')

# Load hcpcsevents and d_hcpcs data
hcpcsevents = pd.read_csv('/data/share/AKI/3.0/hosp/hcpcsevents.csv.gz', compression='gzip')
dhcpc = pd.read_csv('/data/share/AKI/3.0/hosp/d_hcpcs.csv.gz', compression='gzip')

# Merge hcpcsevents with d_hcpcs data
new_hcpcsevents = pd.merge(hcpcsevents, dhcpc, left_on='hcpcs_cd', right_on='code')

# Load data
diagnoses_icd = pd.read_csv('/data/share/AKI/3.0/hosp/diagnoses_icd.csv.gz', compression='gzip')
aki_icds = ['5845', '5846', '5847', '5848', '5849', '66930', '66932', '66934', 'N17', 'N170', 'N171', 'N172', 'N178', 'N179', 'O904']
d_icd_diagnoses = pd.read_csv('/data/share/AKI/3.0/hosp/d_icd_diagnoses.csv.gz', compression='gzip')

# Filter AKI-related hadm_id and subject_id
aki_hadms = diagnoses_icd.loc[diagnoses_icd.icd_code.apply(lambda x: x in aki_icds), 'hadm_id'].unique().tolist()
aki_hadms = set(aki_hadms)

tmp_diagnoses_icd = diagnoses_icd[['subject_id', 'hadm_id']].drop_duplicates()
hadm_to_subject = {str(h): s for s, h in zip(tmp_diagnoses_icd.subject_id.tolist(), tmp_diagnoses_icd.hadm_id.tolist())}
aki_subject = [s for h, s in hadm_to_subject.items() if int(h) in aki_hadms]
aki_subject = set(aki_subject)

# Filter data before AKI occurrence
aki_prev_hadm = []
aki_subject_for_check = []
for key, df in tqdm(tmp_diagnoses_icd.groupby(by=['subject_id'])):
    first_h = list(set(df.hadm_id).intersection(aki_hadms))
    if first_h:
        first_h = np.array(first_h).min()
        prev_h = df.loc[df.hadm_id <= first_h, 'hadm_id'].tolist()
        aki_prev_hadm.extend(prev_h)
        aki_subject_for_check.append(key)

# Filter procedures_icd data
procedures_icd = pd.read_csv('/data/share/AKI/3.0/hosp/procedures_icd.csv.gz', compression='gzip')

def drop_hadms_after_aki(df):
    df = df.loc[~df.hadm_id.isnull(), :]
    df = df.loc[(df.subject_id.apply(lambda x: x not in aki_subject)) | 
                ((df.subject_id.apply(lambda x: x in aki_subject)) * 
                 (df.hadm_id.apply(lambda x: x in aki_prev_hadm))), :]
    n_aki_hadms = len(set(df.hadm_id).intersection(aki_prev_hadm))
    print(f'# unique aki hadms: {n_aki_hadms}\n# unique hadms: {df.hadm_id.nunique()}')
    return df

procedures_icd = drop_hadms_after_aki(procedures_icd)

# Calculate ICD code frequencies
def count_after_groupby_hadm_id(df):
    df_hadm_id = set(df.hadm_id)
    res = []
    print(f'There are {len(df_hadm_id)} hadm_ids in the dataframe')
    for h, h_df in tqdm(df.groupby(by=['subject_id', 'hadm_id'])):
        res.extend(list(set(h_df.icd_code)))
    res = dict(pd.DataFrame(res).value_counts())
    res = {k[0]: v for k, v in res.items()}
    return res

aki_proc_icd = procedures_icd.loc[procedures_icd.hadm_id.apply(lambda x: x in aki_prev_hadm), :]
aki_proc_icd_cnt = count_after_groupby_hadm_id(aki_proc_icd)
proc_icd_cnt = count_after_groupby_hadm_id(procedures_icd)

# Calculate proportions and filter top ICD codes
aki_proc_icd_prop = {icd_code: aki_proc_icd_cnt[icd_code] / proc_icd_cnt[icd_code] for icd_code in aki_proc_icd_cnt.keys()}
aki_proc_icd_top = {k for k, v in sorted(aki_proc_icd_prop.items(), key=lambda x: -x[1]) if v > 0.5}

proc_icd_top = {k for k, v in sorted(proc_icd_cnt.items(), key=lambda x: -x[1]) if v > np.mean(list(proc_icd_cnt.values()))}

aki_proc_icd_top = aki_proc_icd_top.intersection(proc_icd_top)
print(len(aki_proc_icd_top))

# Loading procedures ICD file
procedures_icd = pd.read_csv('/data/share/AKI/3.0/hosp/procedures_icd.csv.gz', compression='gzip')

# Defining top AKI procedure ICD codes
aki_proc_icd_top = ["0014", "0015", "0092", "0093", "02H633Z", "9672", "9971", "B543ZZA", "DP081ZZ", "XW033H5"]

# Loading procedure metadata
d_icd_procedures = pd.read_csv('/data/share/AKI/3.0/hosp/d_icd_procedures.csv.gz', compression='gzip')

# Filtering data by top AKI ICD codes
filtered_procedures = d_icd_procedures.loc[d_icd_procedures.icd_code.apply(lambda x: x in aki_proc_icd_top), :]

# Creating a matrix for procedures ICD frequencies
proc_icd = np.zeros((procedures_icd.hadm_id.nunique(), len(aki_proc_icd_top)))
trow = 0

for group, df in tqdm(procedures_icd.groupby(by=['subject_id', 'hadm_id'])):
    proc_icd_i = dict(df.icd_code.value_counts())
    proc_icd_keys = set(proc_icd_i.keys())
    for idx, aki_proc_icd in enumerate(aki_proc_icd_top):
        if aki_proc_icd in proc_icd_keys:
            proc_icd[trow, idx] = proc_icd_i[aki_proc_icd]
    trow += 1

# Renaming procedure ICD columns
proc_icd_colnames = [f'hosp_procedures_icd_{i}' for i in aki_proc_icd_top]
proc_icd_df = pd.DataFrame(proc_icd, columns=proc_icd_colnames)

# Saving processed ICD data
proc_icd_df.to_csv('hosp/new/procedures_icd.csv', index=False)

# Loading laboratory events data
labevents = pd.read_csv('/data/share/AKI/3.0/hosp/labevents.csv.gz', compression='gzip')

# Dropping NaN values for hospital admission IDs
labevents = labevents.dropna(subset=['hadm_id'])

# Loading transfer data
transfers = pd.read_csv('/data/share/AKI/3.0/hosp/transfers.csv.gz', compression='gzip')

# Saving the final dataframe
labevents.to_csv('hosp/new/labevents_clean.csv', index=False)

# Data Filtering
labevents = labevents.loc[labevents.subject_id == 10000032, :]

# Sorting and Formatting Data
a = transfers.loc[(transfers.subject_id == 10000032) & (transfers.eventtype == 'admit'), ['hadm_id', 'intime']]
a['intime'] = a['intime'].apply(lambda x : convert_to_datetime(x))
a.sort_values(by='intime', inplace=True)

# Helper Function
def find_hadm(subject, charttime):
    hadm_times = transfers.loc[(transfers.subject_id == subject) & (transfers.eventtype == 'admit'), ['hadm_id', 'intime']]
    hadm_times['intime'] = hadm_times.intime.apply(lambda x : convert_to_datetime(x))
    hadm_times.sort_values(by='intime', inplace=True)
    res = hadm_times.iloc[0, 0]
    for i in range(len(hadm_times)):
        if hadm_times.iloc[i, 1] < charttime:
            res = hadm_times.iloc[i, 0]
        else:
            return res
    return res

# Test the Function
find_hadm(10000032, convert_to_datetime('2180-03-23 11:51:00'))
find_hadm(10000032, convert_to_datetime('2180-06-28 12:00:00'))
find_hadm(10000032, convert_to_datetime('2180-12-01 12:00:00'))

# Filtering and Grouping Data
labevents = labevents.loc[~labevents.hadm_id.isnull(), :]
aki_itemids = [50912, 51006, 50971, 50983, 50902, 50882, 50868, 50931, 51221, 51265, 51222, 51301, 51249, 51248, 51250, 51279, 51277, 50960, 50893, 50970]

# Creating Feature Matrices
n_hadms = len(labevents.hadm_id.unique())
did_itemid = np.zeros((n_hadms, len(aki_itemids)))
is_flag = np.zeros((n_hadms, len(aki_itemids)))

# Grouping and Analyzing Data
for group, df in tqdm(labevents.groupby(by=['subject_id', 'hadm_id'])):
    tmp_itemids = set(df.itemid)
    for idx, tmp_itemid in enumerate(aki_itemids):
        if tmp_itemid in tmp_itemids:
            did_itemid[trow, idx] = 1
            tmp_flag = (~df.loc[df.itemid == tmp_itemid, 'flag'].isnull()).any().item()
            is_flag[trow, idx] = tmp_flag
    trow += 1

# Creating DataFrames for Feature Matrices
did_itemid_colnames = [f'hosp_itemid_{i}' for i in aki_itemids]
is_flag_colnames = [f'hosp_itemid_is_flag_{i}' for i in aki_itemids]

did_itemid = pd.DataFrame(did_itemid, columns=did_itemid_colnames)
is_flag = pd.DataFrame(is_flag, columns=is_flag_colnames)

# Combining Results with Original Data
new_labevents = labevents[['subject_id', 'hadm_id']].drop_duplicates().reset_index(drop=True)
new_labevents = pd.concat([new_labevents, did_itemid, is_flag], axis=1)

# Function to drop HADM_IDs after AKI
def drop_hadms_after_aki(df):
    df = df.loc[~df.hadm_id.isnull(), :]
    df = df.loc[(df.subject_id.apply(lambda x: x not in aki_subject)) | 
                ((df.subject_id.apply(lambda x: x in aki_subject)) & 
                 (df.hadm_id.apply(lambda x: x in aki_prev_hadm))), :]
    n_aki_hadms = len(set(df.hadm_id).intersection(aki_prev_hadm))
    print(f'# unique aki hadms : {n_aki_hadms}\n# unique hadms : {df.hadm_id.nunique()}')
    return df

# Function to count prescriptions after grouping by HADM_ID
def count_after_groupby_hadm_id(df):
    df_hadm_id = set(df.hadm_id)
    res = []
    print(f'There are {len(df_hadm_id)} HADM_IDs in the dataframe')
    for h, h_df in tqdm(df.groupby(by=['subject_id', 'hadm_id'])):
        res.extend(list(set(h_df.ndc)))
    res = dict(pd.DataFrame(res).value_counts())
    res = {k[0]: v for k, v in res.items()}
    return res

# Load the prescriptions dataset
prescriptions = pd.read_csv('/data/share/AKI/3.0/hosp/prescriptions.csv.gz', compression='gzip')

# Process prescriptions to drop HADM_IDs after AKI
prescriptions = drop_hadms_after_aki(prescriptions)
prescriptions = prescriptions.loc[~prescriptions.ndc.isnull(), :]

# Extract AKI-related prescriptions
aki_prescriptions = prescriptions.loc[prescriptions.hadm_id.apply(lambda x: x in aki_prev_hadm), :]

# Count prescription occurrences
aki_prescriptions_cnt = count_after_groupby_hadm_id(aki_prescriptions)
prescriptions_cnt = count_after_groupby_hadm_id(prescriptions)

# Calculate proportion of AKI prescriptions
aki_prescriptions_prop = dict()
for prescription in tqdm(list(aki_prescriptions_cnt.keys())):
    aki_prescriptions_prop[prescription] = aki_prescriptions_cnt[prescription] / prescriptions_cnt[prescription]

# Filter top prescriptions based on proportions
aki_prescriptions_top = dict(sorted(aki_prescriptions_prop.items(), key=lambda x: -x[1]))
aki_prescriptions_top = {k for k, v in aki_prescriptions_top.items() if v > 0.5}

# Filter prescriptions with high overall usage
prescriptions_top = dict(sorted(prescriptions_cnt.items(), key=lambda x: -x[1]))
mean_prescriptions_top_val = np.array(list(prescriptions_top.values())).mean()
prescriptions_top = {k for k, v in prescriptions_top.items() if v > mean_prescriptions_top_val}

# Intersect the top prescriptions
aki_prescriptions_top = aki_prescriptions_top.intersection(prescriptions_top)

# Create a prescription matrix
pres = np.zeros((prescriptions.hadm_id.nunique(), len(aki_prescriptions_top)))
trow = 0
for group, df in tqdm(prescriptions.groupby(by=['subject_id', 'hadm_id'])):
    pres_i = dict(df.ndc.value_counts())
    pres_keys = set(pres_i.keys())
    for idx, aki_prescription in enumerate(aki_prescriptions_top):
        if aki_prescription in pres_keys:
            pres[trow, idx] = pres_i[aki_prescription]
    trow += 1

# Create a DataFrame with prescription data
pres_colnames = [f'hosp_prescriptions_{i}' for i in aki_prescriptions_top]
pres = pd.DataFrame(pres, columns=pres_colnames)

# Merge with subject and HADM_ID information
new_prescriptions = prescriptions[['subject_id', 'hadm_id']].drop_duplicates().reset_index(drop=True)
final_prescriptions = pd.concat([new_prescriptions, pres], axis=1)

# Save the processed data
new_data_dir = './hosp/new'
if not os.path.exists(new_data_dir):
    os.makedirs(new_data_dir)
final_prescriptions.to_csv(f'{new_data_dir}/processed_prescriptions.csv', index=False)

# Load microbiology data with gzip compression
microbiology = pd.read_csv('/data/share/AKI/3.0/hosp/microbiologyevents.csv.gz', compression='gzip')

# Filter out rows where org_itemid is NaN
microbiology = microbiology.loc[~microbiology.org_itemid.isnull(), :]

# Select specific columns from the microbiology DataFrame
microbiology = microbiology[['subject_id', 'hadm_id', 'test_itemid', 'org_itemid', 'ab_itemid', 'dilution_value', 'interpretation']]

# Filter rows where ab_itemid is not null
microbiology = microbiology.loc[~microbiology.ab_itemid.isnull(), :]

# Create a composite column for unique test identifiers
microbiology['test_org_ab_itemid'] = (
    microbiology['test_itemid'].apply(lambda x: str(int(x))) + '/' +
    microbiology['org_itemid'].apply(lambda x: str(int(x))) + '/' +
    microbiology['ab_itemid'].apply(lambda x: str(int(x)))
)

# Filter for AKI-related microbiology data
aki_microbiology = microbiology.loc[
    microbiology.hadm_id.apply(lambda x: x in aki_prev_hadm), :
]

# Count the occurrences of unique test identifiers grouped by hadm_id
def count_after_groupby_hadm_id(df):
    df_hadm_id = set(df.hadm_id)
    res = []
    for h, h_df in tqdm(df.groupby(by=['subject_id', 'hadm_id'])):
        res.extend(list(set(h_df.test_org_ab_itemid)))
    res = dict(pd.DataFrame(res).value_counts())
    res = {k[0]: v for k, v in res.items()}
    return res

aki_microbiology_cnt = count_after_groupby_hadm_id(aki_microbiology)
microbiology_cnt = count_after_groupby_hadm_id(microbiology)

# Calculate proportion of AKI-specific microbiology tests
aki_microbiology_prop = {
    i: aki_microbiology_cnt[i] / microbiology_cnt[i]
    for i in aki_microbiology_cnt.keys()
}

# Identify the top AKI-related microbiology tests
aki_microbiology_top = dict(
    sorted(aki_microbiology_prop.items(), key=lambda x: -x[1])
)
aki_microbiology_top = {k for k, v in aki_microbiology_top.items() if v > 0.6}

# Identify microbiology tests with occurrences above the mean
mean_microbiology_top_val = np.array(list(microbiology_cnt.values())).mean()
microbiology_top = {k for k, v in microbiology_cnt.items() if v > mean_microbiology_top_val}

# Filter tests that are both AKI-related and frequently occurring
aki_microbiology_top = aki_microbiology_top.intersection(microbiology_top)

# Create a matrix for microbiology test results
mic = np.zeros((microbiology.hadm_id.nunique(), len(aki_microbiology_top)))
trow = 0
for group, df in tqdm(microbiology.groupby(by=['subject_id', 'hadm_id'])):
    mic_i = df[['test_org_ab_itemid', 'interpretation']]
    mic_keys = set(mic_i.test_org_ab_itemid)
    for idx, tmp_mic in enumerate(aki_microbiology_top):
        if tmp_mic in mic_keys:
            val = mic_i.loc[mic_i.test_org_ab_itemid == tmp_mic, 'interpretation'].tolist()[0]
            if val == 'S':
                val = 1
            elif val == 'I':
                val = 2
            elif val == 'R':
                val = 3
            else:
                val = 0
            mic[trow, idx] = val
    trow += 1

# Processing microbiology data
mic_colnames = [f'microbiology_{i}' for i in aki_microbiology_top]  # Generate new column names
mic = pd.DataFrame(mic)  # Convert mic to a DataFrame
mic.columns = mic_colnames  # Assign the new column names

# Remove duplicates and reset the index for the existing microbiology DataFrame
new_microbiology = microbiology[['subject_id', 'hadm_id']].drop_duplicates().reset_index(drop=True)

# Combine the new columns and save to CSV
new_microbiology = pd.concat((new_microbiology, mic), axis=1)
new_microbiology.to_csv('hosp/new/microbiologyevents.csv')

# Processing patients data
patients = pd.read_csv('/data/share/AKI/3.0/hosp/patients.csv.gz', compression='gzip')  # Load the data
patients = patients[['subject_id', 'gender', 'anchor_age', 'dod']]  # Select required columns

# Add a new column indicating whether the patient has died
patients['died'] = ~patients.dod.isnull()

# Drop unnecessary columns and verify
patients = patients.drop('dod', axis=1)  # Drop the 'dod' column
patients.isnull().sum()  # Check for missing values

# Save the processed data to CSV
patients.to_csv('hosp/new/patients.csv')